# 2018-09-04 (Martes)

---

## Descarga librerias

```
gensim
    (Anaconda) conda install -c anaconda gensim
    (Mac o Linux) pip install --upgrade gensim

actualizar nltk
    nltk.download()
```

In [1]:
import nltk

In [ ]:
nltk.download()

## Datos no estructurados

Con datos no estructurado me refiero a cualquier colección de información que no esté en formato de base de datos y sobre la cual queramos extraer información relevante para nuestro análisis. En las siguientes clases trabajaremos extrayendo información de 3 tipos:

- HTML
- Texto (usando Expresiones Regulares)
- APIs

## 1. HTML

HTML (Hypertext Markup Language) es el lenguaje estándar de páginas web. Dicho lenguaje permite estructurar y formatear documentos de texto a los cuales accedemos usualmente a través del explorador de internet (Chrome, Firefox, IExplorer, etc).

La estructura típica de un documento HTML es la siguiente:

```html
<!DOCTYPE html>
<html>
    <head>
    <title>Page Title</title>
    </head>
<body>
    <div>
        <h1>My First Heading</h1>
        <p>My first paragraph.</p>
    </div>
</body>
</html>
```


Para hacer solicitudes a páginas web, utilizaremos el paquete "requests"

Documentación: http://docs.python-requests.org/en/master/

### Ejemplo 1. Búsqueda información "I'm feeling lucky"

In [6]:
import requests

base_url = 'http://www.google.com/search?q={}&btnI'

terminos_a_buscar = ['carlos', 'alvarado']

url_google = base_url.format('+'.join(terminos_a_buscar))

print(url_google)

request1 = requests.get(url_google)

http://www.google.com/search?q=carlos+alvarado&btnI


Hm.. interesante.. seleccionemos algunos links de esa página y saquemos más información relevante..

In [7]:
#se supone que puedo automatizar esto para buscar links relevantes..
urls = [url_google,
        'https://www.cidob.org/biografias_lideres_politicos/america_central_y_caribe/costa_rica/carlos_alvarado_quesada',
        'https://es.wikipedia.org/wiki/Costa_Rica',
        'https://elpais.com/internacional/2018/05/07/america/1525706014_193325.html'
       ]

#objeto para guardar los resultados de busqueda
resultados = []

#ahora iteramos para las URLs
for url in urls:
    request_data = requests.get(url)
    resultados.append(request_data)


Existen librerías de Python que nos permiten "navegar" a través de la estructura HTML (a través de "tags") y extraer información relevante. 

La librería que usaremos aquí se llama "BeautifulSoup".

Documentación: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Tutorial webscraping con BeautifulSoup: https://www.dataquest.io/blog/web-scraping-tutorial-python/

In [13]:
#ahora automaticemos la extraccion de texto de estas dos páginas

from bs4 import BeautifulSoup

textos_limpios = []

for request in resultados:
    soup = BeautifulSoup(request.content, 'html.parser')

    body = soup.select('body')

    texto = body[0].get_text()
    
    #"texto" viene con muchos caracteres "\n". Eliminemos y separemos con .
    
    texto_limpio = texto.replace('\n', '. ')

#     texto_limpio = ' '.join([linea.strip() + '.' for linea in texto.split('\n')])
    
    textos_limpios.append(texto_limpio)
    

In [14]:
textos_limpios[0]

' . . . . . . Carlos Alvarado Quesada . De Wikipedia, la enciclopedia libre .  Ir a la navegación. Ir a la búsqueda. Carlos Alvarado Quesada. Carlos Alvarado Quesada (2018). 48º Presidente de la República de Costa Rica  Actualmente en el cargo. Desde el 8 de mayo de 2018Gabinete. Gabinete de Carlos Alvarado QuesadaPredecesor. Luis Guillermo Solís RiveraVicepresidenta Primera. Epsy Campbell BarrVicepresidente Segundo. Marvin Rodríguez Cordero. Ministro de Trabajo y Seguridad Social de Costa Rica. 29 de marzo de 2016-19 de enero de 2017Presidente. Luis Guillermo Solís RiveraPredecesor. Víctor Morales MoraSucesor. Alfredo Hasbum Camacho. Ministro de Desarrollo Humano e Inclusión Social. 10 de julio de 2014-29 de marzo de 2016Presidente. Luis Guillermo Solís RiveraPredecesor. Fernando Marín RojasSucesor. Emilio Arias Rodríguez. Información personalNombre en español. Carlos Andrés Alvarado Quesada Nacimiento. 14 de enero de 1980 (38\xa0años) Pavas, San José,  Costa RicaNacionalidad. Costarr

¿como podemos extraer info desde aqui? Opcion 1. Natural Language Processing (NLP)

In [15]:
import nltk

In [16]:
textos = ' '.join(textos_limpios)

palabras_de_textos = [word for word in textos.split()]

nltk_text = nltk.Text(palabras_de_textos)

In [43]:
print(nltk_text.collocations())

fdist1 = nltk.FreqDist(nltk_text) 
fdist1.most_common(10)


Costa Rica; Costa Rica.; Artículo principal:; Archivado desde; Costa
Rica,; Palmer, 2011,; requiere |url=; |fechaacceso= requiere; Botey
Sobrado,; Costa Rica».; |url= (ayuda).; Nación. Consultado; Sobrado,
2002,; (en inglés).; «Costa Rica; América Latina; Costa Rica..; Carlos
Alvarado; Estados Unidos; así como
None


[('de', 3913),
 ('.', 2884),
 ('la', 1429),
 ('el', 1416),
 ('y', 1265),
 ('en', 1083),
 ('del', 627),
 ('a', 547),
 ('Costa', 540),
 ('que', 486)]

hmm.. Lo anterior no es muy útil porque tiene muchos "stopwords"... saquémoslos

In [44]:
from nltk.corpus import stopwords
palabras_stop = stopwords.words('spanish')

palabras_de_textos_limpios = [word for word in textos.split() if word not in palabras_stop]

nltk_text_limpio = nltk.Text(palabras_de_textos_limpios)

nltk_text_limpio.collocations()

fdist2 = nltk.FreqDist(nltk_text_limpio) 
fdist2.most_common(50)

Costa Rica; Costa Rica.; Archivado original; Artículo principal:;
Molina Palmer,; Costa Rica,; Palmer, 2011,; requiere |url=;
|fechaacceso= requiere; Botey Sobrado,; |url= (ayuda).; Costa Rica».;
Nación. Consultado; Sobrado, 2002,; (en inglés).; «Costa Rica; América
Latina; Costa Rica..; enero 2012.; Carlos Alvarado


[('.', 2884),
 ('Costa', 540),
 ('↑', 398),
 ('Rica', 296),
 ('La', 250),
 ('Consultado', 245),
 ('El', 190),
 ('país', 145),
 ('=', 132),
 ('En', 120),
 ('Rica.', 112),
 ('p.', 94),
 ('San', 81),
 ('Nacional', 77),
 ('2015.', 64),
 ('agosto', 62),
 ('}.', 62),
 ('enero', 60),
 ('Rica,', 60),
 ('{.', 60),
 ('2017.', 59),
 ('Alvarado', 58),
 ('if', 54),
 ('Carlos', 53),
 ('abril', 53),
 ('América', 53),
 ('mayo', 52),
 ('2016.', 52),
 ('Los', 50),
 ('marzo', 49),
 ('%', 49),
 ('b', 48),
 ('Archivado', 45),
 ('José', 44),
 ('Nación.', 43),
 ('original', 43),
 ('septiembre', 42),
 ('Universidad', 41),
 ('años', 41),
 ('«Costa', 40),
 ('Rica».', 40),
 ('1', 39),
 ('febrero', 39),
 ('diciembre', 39),
 ('var', 39),
 ('costarricense', 38),
 ('Artículo', 38),
 ('Instituto', 37),
 ('primera', 37),
 ('siglo', 37)]

Hm... hagamos una funcion para limpiar algunas palabras


In [19]:
def helper_clean_word(w):
        
    del_chars = ['«', ',', '↑', '-', '0', 
                 '1', '2', '3', '4', '5', 
                 '6', '7', '8', '9', '#',
                '&', '$', '(', ')', '.',
                '%', '/', ':', ';', '[',
                ']', ]
        
    for c in del_chars:
        w = w.replace(c, '')
        
    w = w.replace('á', 'a')
    w = w.replace('é', 'e')
    w = w.replace('í', 'i')
    w = w.replace('ó', 'o')
    w = w.replace('ú', 'u')
       
    return w

In [45]:
palabras_de_textos_limpios2 = [helper_clean_word(word) for word in textos.split() if word not in palabras_stop and helper_clean_word(word)]

nltk_text_limpio2 = nltk.Text(palabras_de_textos_limpios2)

nltk_text_limpio2.collocations()

fdist3 = nltk.FreqDist(nltk_text_limpio2) 
fdist3.most_common(50)

Costa Rica; Archivado original; Molina Palmer; San Jose; Articulo
principal; Estados Unidos; America Latina; requiere |url=;
|fechaacceso= requiere; Botey Sobrado; |url= ayuda; Costa Rica»;
Carlos Alvarado; Consultado agosto; Nacion Consultado; Valle Central;
Consultado enero; Luis Guillermo; Asamblea Legislativa; Consultado
febrero


[('Costa', 584),
 ('Rica', 518),
 ('La', 274),
 ('Consultado', 245),
 ('pais', 206),
 ('El', 199),
 ('=', 132),
 ('En', 121),
 ('p', 98),
 ('Nacional', 92),
 ('}', 85),
 ('San', 84),
 ('Alvarado', 78),
 ('Jose', 78),
 ('costarricense', 71),
 ('nacional', 70),
 ('Carlos', 67),
 ('{', 67),
 ('años', 66),
 ('agosto', 66),
 ('\u200b', 65),
 ('America', 65),
 ('Nacion', 62),
 ('enero', 61),
 ('abril', 58),
 ('mayo', 55),
 ('marzo', 54),
 ('if', 54),
 ('principal', 53),
 ('Los', 52),
 ('Pacifico', 51),
 ('b', 48),
 ('Universidad', 46),
 ('septiembre', 46),
 ('Central', 46),
 ('Archivado', 45),
 ('Republica', 44),
 ('diciembre', 44),
 ('original', 43),
 ('Rica»', 42),
 ('var', 41),
 ('febrero', 40),
 ('Estado', 39),
 ('primera', 39),
 ('Articulo', 39),
 ('Molina', 39),
 ('siglo', 38),
 ('Instituto', 37),
 ('poblacion', 37),
 ('Juan', 37)]

Ahora podemos tratar de buscar informacion relevante... usemos el modelo "Word2Vec"

In [21]:
from gensim.models import Word2Vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [37]:
from nltk.tokenize import sent_tokenize

sent_tokenize_list = []

clean_sentences = []

for t in textos_limpios:
    
    # Primero, separamos cada texto en oraciones
    
    #sentences = t.split('.')    
    sentences = sent_tokenize(t)
    
    for s in sentences:
    
        # Ahora, para cada oración, separamos las palabras y las limpiamos
        clean_sentence = []
        for w in s.split(' '):
            
            # Aqui ocupamos nuestra función para limpiar.
            clean_word = helper_clean_word(w)
            
            # Cuando se procesa texto, también se suele "homogeneizar" o "estandarizar"
            # Esto se llama "stemming" y "lemmatization"
            
            if len(clean_word) > 2 and clean_word not in palabras_stop:
                clean_sentence.append(clean_word)
        
        if len(clean_sentence) > 3:
            clean_sentences.append(clean_sentence)



*Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word*

In [38]:
# con "clean_sentences" podemos entrenar nuestro modelo

b = Word2Vec(clean_sentences, size = 50)


In [39]:
len(clean_sentences)

1593

In [ ]:
print(clean_sentences)

In [25]:
len(b.wv.vocab)

865

In [26]:
b.wv.vocab

{'"undefined"': <gensim.models.keyedvectors.Vocab at 0x28b068b50f0>,
 'Accion': <gensim.models.keyedvectors.Vocab at 0x28b05457208>,
 'Actualmente': <gensim.models.keyedvectors.Vocab at 0x28b05454898>,
 'Ademas': <gensim.models.keyedvectors.Vocab at 0x28b068887b8>,
 'Aeropuerto': <gensim.models.keyedvectors.Vocab at 0x28b068a9908>,
 'Airlines': <gensim.models.keyedvectors.Vocab at 0x28b068af2e8>,
 'Alajuela': <gensim.models.keyedvectors.Vocab at 0x28b068a06a0>,
 'Alemania': <gensim.models.keyedvectors.Vocab at 0x28b068a9a20>,
 'Alvarado': <gensim.models.keyedvectors.Vocab at 0x28b04d056a0>,
 'America': <gensim.models.keyedvectors.Vocab at 0x28b0688b3c8>,
 'Ana': <gensim.models.keyedvectors.Vocab at 0x28b06886d30>,
 'Analisis': <gensim.models.keyedvectors.Vocab at 0x28b0688d748>,
 'Andres': <gensim.models.keyedvectors.Vocab at 0x28b05457550>,
 'Archive': <gensim.models.keyedvectors.Vocab at 0x28b0688d0f0>,
 'Arias': <gensim.models.keyedvectors.Vocab at 0x28b06895860>,
 'Articulo': <gens

In [ ]:
dir(b.vocabulary)

In [40]:
b.wv.most_similar('Carlos')

[('mas', 0.986006498336792),
 ('gran', 0.9851000308990479),
 ('Costa', 0.9848820567131042),
 ('pais', 0.9838680624961853),
 ('Rica', 0.9836851358413696),
 ('Jose', 0.9834976196289062),
 ('costarricense', 0.9834609031677246),
 ('cuenta', 0.9828611612319946),
 ('nacional', 0.9821909666061401),
 ('Nicaragua', 0.9820826053619385)]

In [41]:
b.wv.most_similar(positive=['Carlos', 'presidente'])

[('Costa', 0.9935423135757446),
 ('Rica', 0.9926671981811523),
 ('mas', 0.9920509457588196),
 ('Jose', 0.9917246699333191),
 ('pais', 0.991149365901947),
 ('Pacifico', 0.9901591539382935),
 ('gran', 0.9901536703109741),
 ('nacional', 0.9899576902389526),
 ('costarricense', 0.9895353317260742),
 ('Nacional', 0.9887441992759705)]

In [32]:
b.wv.similarity('Carlos', 'casa')

0.7538715035895389

#### Ejemplo 2. Puntaje alumnos en Kattis


In [2]:
# es importante correr esta celda solo 1 vez, para evitar hacer 
# muchos GET requests a la pagina de Kattis

import requests

base_url = 'https://open.kattis.com/universities/uc.cl'

# el metodo "get" de la libreria requests nos permite hacer una solicitud "get"
# al servidor señalado. Dicha libreria nos entrega un objeto "Response",
# que incluye información sobre la respuesta que tuvo nuestra solicitud

request = requests.get(base_url)

print(request.text)

<!DOCTYPE html>

<html lang="en">
<head>
    <meta charset="UTF-8" >
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Pontifical Catholic University of Chile &ndash; Kattis, Kattis</title>

    <link href="https://cdnjs.cloudflare.com/ajax/libs/jqueryui/1.10.3/css/base/minified/jquery-ui.min.css" rel="stylesheet">

    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.10.1/jquery.min.js"></script>
    <script src="//ajax.googleapis.com/ajax/libs/jqueryui/1.10.3/jquery-ui.min.js"></script>

    <!-- Fonts/Icons -->
    <link href="//cdnjs.cloudflare.com/ajax/libs/font-awesome/4.0.1/css/font-awesome.min.css" rel="stylesheet">

    <link href="//fonts.googleapis.com/css?family=Open+Sans:400,300,300italic,400italic,600,600italic,700,800,700italic,800italic%7CMerriweather:400,400italic,700" rel="stylesheet" type="text/css">

    <!-- Bootstrap CSS -->
    <link href="//cdnjs.cloudflare.com/ajax/

In [3]:
from bs4 import BeautifulSoup

# El objeto "BeautifulSoup", toma como parámetros el texto de la página HTML
# y el tipo de "parser" (traductor o codificador) usar

soup = BeautifulSoup(request.content, 'html.parser')

#ahora, el objeto "soup" nos permite buscar tags dentro del HTML
tags = soup.find_all('table')

#tags es una lista de objetos "Tag" (este es similar a un objeto "soup")
print(tags)

for tag in tags:
    print(type(tag))

print(type(soup))

[<table>
<tr>
<td><span>Rank</span></td>
<td><span>Score</span></td>
</tr>
<tr>
<td>161</td>
<td>129.9</td>
</tr>
</table>, <table class="table table-hover table-kattis">
<thead>
<tr>
<th class="table-min-wrap">#</th>
<th>User</th>
<th>Country</th>
<th class="table-min-wrap">Score <span class="wrapped-link"><a aria-label="Explanation of score" href="/help/ranklist#score">?</a></span></th>
</tr>
</thead>
<tbody>
<tr class="">
<td>
                                                1
                                            </td>
<td><a href="/users/martins">Martín Muñoz Cruces</a></td>
<td class="table-min-wrap">
<div class="country-flag has_flag"><a href="/countries/CHL" title="Chile"><img src="/images/countries/CHL/40px?e3bcbe8bc7c9f199da895db8e5f03f20"/></a><span class="text"><a href="/countries/CHL">Chile</a></span></div>
</td>
<td class="table-td-align-right">448.4</td>
</tr>
<tr class="">
<td>
                                                2
                                      

In [ ]:
help(BeautifulSoup)

In [ ]:
score_table = tags[1]

print(score_table)

In [ ]:
#cargo lista de usuarios a buscar
with open('../clase7_20180830/usuarios_kattis.txt', 'r', encoding='utf-8-sig') as archivo:
    alumnos = [line.strip() for line in archivo]

print('Lista de usuarios a buscar:')
print(alumnos)
print()

#selecciono las filas de la tabla
row_tags = score_table.select('tr')
print('Hay {} filas en la tabla'.format(len(row_tags)))

In [ ]:

print('Descargando datos de la tabla:...')
print()

DATA = []

#extraigo información fila por fila
for row in row_tags:
    
    #selecciono las columnas de esa fila
    col_tags = row.select('td')
    
    #extraigo el texto de cada "celda"
    aux_row = [celda.get_text().strip() for celda in col_tags]
    
    #si hay información, la guardo
    if aux_row:
        DATA.append(aux_row)

for row in DATA:
    print(row)


In [ ]:
#extraigo información relevante de los usuarios que me interesan
for num, nombre, pais, puntaje in DATA:
    if nombre in alumnos:
        print((num, nombre, pais, puntaje))